# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 8 2023 12:38PM,256946,10,PRF-52256,Bio-PRF,Released
1,Feb 8 2023 12:38PM,256946,10,PRF-52262,Bio-PRF,Released
2,Feb 8 2023 12:38PM,256946,10,PRF-52266,Bio-PRF,Released
3,Feb 8 2023 12:38PM,256946,10,PRF-52273,Bio-PRF,Released
4,Feb 8 2023 12:38PM,256946,10,PRF-52281,Bio-PRF,Released
5,Feb 8 2023 12:38PM,256946,10,PRF-52286,Bio-PRF,Released
6,Feb 8 2023 12:38PM,256946,10,PRF-52291,Bio-PRF,Released
7,Feb 8 2023 12:38PM,256946,10,PRF-52321,Bio-PRF,Released
8,Feb 8 2023 12:38PM,256946,10,PRF-52324,Bio-PRF,Released
9,Feb 8 2023 12:38PM,256946,10,PRF-52334,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
47,256942,Released,1
48,256943,Released,4
49,256946,Released,15
50,256947,Released,1
51,256948,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256942,NaN,NaN,1.0
256943,NaN,NaN,4.0
256946,NaN,NaN,15.0
256947,NaN,NaN,1.0
256948,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,37.0,8.0,7.0
256826,0.0,0.0,113.0
256827,12.0,1.0,3.0
256828,1.0,7.0,0.0
256857,0.0,0.0,11.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256783,37,8,7
256826,0,0,113
256827,12,1,3
256828,1,7,0
256857,0,0,11


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,37,8,7
1,256826,0,0,113
2,256827,12,1,3
3,256828,1,7,0
4,256857,0,0,11


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256783,37,8,7
1,256826,,,113
2,256827,12,1,3
3,256828,1,7,
4,256857,,,11


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 8 2023 12:38PM,256946,10,Bio-PRF
15,Feb 8 2023 12:38PM,256948,10,Emerginnova
16,Feb 8 2023 12:34PM,256947,10,Bio-PRF
17,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation
21,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd"
22,Feb 8 2023 12:16PM,256940,10,Emerginnova
25,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC"
26,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.
28,Feb 8 2023 11:55AM,256936,10,"Nextsource Biotechnology, LLC"
30,Feb 8 2023 11:48AM,256931,15,DXM Pharmaceutical Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 8 2023 12:38PM,256946,10,Bio-PRF,,,15
1,Feb 8 2023 12:38PM,256948,10,Emerginnova,,,1
2,Feb 8 2023 12:34PM,256947,10,Bio-PRF,,,1
3,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,,,4
4,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",,,1
5,Feb 8 2023 12:16PM,256940,10,Emerginnova,,,3
6,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",,,1
7,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.,,,2
8,Feb 8 2023 11:55AM,256936,10,"Nextsource Biotechnology, LLC",,,2
9,Feb 8 2023 11:48AM,256931,15,DXM Pharmaceutical Inc.,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15,,
1,Feb 8 2023 12:38PM,256948,10,Emerginnova,1,,
2,Feb 8 2023 12:34PM,256947,10,Bio-PRF,1,,
3,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4,,
4,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1,,
5,Feb 8 2023 12:16PM,256940,10,Emerginnova,3,,
6,Feb 8 2023 12:07PM,256939,10,"ClearSpec, LLC",1,,
7,Feb 8 2023 12:07PM,256938,19,VITABIOTICS USA INC.,2,,
8,Feb 8 2023 11:55AM,256936,10,"Nextsource Biotechnology, LLC",2,,
9,Feb 8 2023 11:48AM,256931,15,DXM Pharmaceutical Inc.,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15,,
1,Feb 8 2023 12:38PM,256948,10,Emerginnova,1,,
2,Feb 8 2023 12:34PM,256947,10,Bio-PRF,1,,
3,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4,,
4,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15.0,NaN,NaN
1,Feb 8 2023 12:38PM,256948,10,Emerginnova,1.0,NaN,NaN
2,Feb 8 2023 12:34PM,256947,10,Bio-PRF,1.0,NaN,NaN
3,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4.0,NaN,NaN
4,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 8 2023 12:38PM,256946,10,Bio-PRF,15.0,0.0,0.0
1,Feb 8 2023 12:38PM,256948,10,Emerginnova,1.0,0.0,0.0
2,Feb 8 2023 12:34PM,256947,10,Bio-PRF,1.0,0.0,0.0
3,Feb 8 2023 12:24PM,256943,10,ISDIN Corporation,4.0,0.0,0.0
4,Feb 8 2023 12:23PM,256942,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5395194,77.0,10.0,3.0
15,1284271,142.0,25.0,49.0
16,513784,1.0,1.0,0.0
19,2568997,23.0,23.0,5.0
20,256930,1.0,0.0,0.0
21,256870,1.0,0.0,0.0
22,256918,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5395194,77.0,10.0,3.0
1,15,1284271,142.0,25.0,49.0
2,16,513784,1.0,1.0,0.0
3,19,2568997,23.0,23.0,5.0
4,20,256930,1.0,0.0,0.0
5,21,256870,1.0,0.0,0.0
6,22,256918,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,77.0,10.0,3.0
1,15,142.0,25.0,49.0
2,16,1.0,1.0,0.0
3,19,23.0,23.0,5.0
4,20,1.0,0.0,0.0
5,21,1.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,77.0
1,15,Released,142.0
2,16,Released,1.0
3,19,Released,23.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,3.0,49.0,0.0,5.0,0.0,0.0,0.0
Executing,10.0,25.0,1.0,23.0,0.0,0.0,0.0
Released,77.0,142.0,1.0,23.0,1.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,3.0,49.0,0.0,5.0,0.0,0.0,0.0
1,Executing,10.0,25.0,1.0,23.0,0.0,0.0,0.0
2,Released,77.0,142.0,1.0,23.0,1.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,3.0,49.0,0.0,5.0,0.0,0.0,0.0
1,Executing,10.0,25.0,1.0,23.0,0.0,0.0,0.0
2,Released,77.0,142.0,1.0,23.0,1.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()